In [14]:
import concurrent.futures
import pandas as pd
def parse_variables(lines):
    variables = {}
    for line in lines:
        if line.startswith("PROBLEM NAME:"):
            variables['problem_name'] = line.split(':')[1].strip()
        elif line.startswith("KNAPSACK DATA TYPE:"):
            variables['knapsack_data_type'] = line.split(':')[1].strip()
        elif line.startswith("DIMENSION:"):
            variables['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith("NUMBER OF ITEMS:"):
            variables['num_items'] = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY OF KNAPSACK:"):
            variables['knapsack_capacity'] = int(line.split(':')[1].strip())
        elif line.startswith("MIN SPEED:"):
            variables['min_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("MAX SPEED:"):
            variables['max_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("RENTING RATIO:"):
            variables['renting_ratio'] = float(line.split(':')[1].strip())
        elif line.startswith("EDGE_WEIGHT_TYPE:"):
            variables['edge_weight_type'] = line.split(':')[1].strip()
    return variables

def parse_node_coord(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        parts = line.split()
        index = int(parts[0])
        x = float(parts[1])
        y = float(parts[2])
        data.append({'Index': index, 'X': x, 'Y': y})
    return pd.DataFrame(data)

def parse_item_section(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, profit, weight, assigned_node = map(int, line.split())
        data.append({'Index': index, 'Profit': profit, 'Weight': weight, 'Assigned_Node': assigned_node})
    return pd.DataFrame(data)

def parse_file_parallel(lines, dimension):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_node_coord = executor.submit(parse_node_coord, lines, 10, dimension)
        future_profit_matrix = executor.submit(parse_item_section, lines, 10 + dimension + 1, dimension)

    node_coord = future_node_coord.result()
    profit_matrix = future_profit_matrix.result()

    return node_coord, profit_matrix


# Example usage:

with open('test-example-n4.txt', 'r') as file:
    lines = file.readlines()

variables = parse_variables(lines)
node_coord, profit_matrix = parse_file_parallel(lines, variables['dimension'])

print("Variables:", variables)
print("Node Coord:", node_coord)
print("Profit Matrix:", profit_matrix)


Variables: {'problem_name': 'Test', 'knapsack_data_type': 'unknown', 'dimension': 4, 'num_items': 3, 'knapsack_capacity': 80, 'min_speed': 0.1, 'max_speed': 1.0, 'renting_ratio': 1.516, 'edge_weight_type': 'CEIL_2D'}
Node Coord:    Index    X    Y
0      1  0.0  0.0
1      2  4.0  0.0
2      3  8.0  3.0
3      4  0.0  3.0
Profit Matrix:    Index  Profit  Weight  Assigned_Node
0      2      34      30              2
1      3      40      40              3
2      4      25      21              4


In [15]:
variables

{'problem_name': 'Test',
 'knapsack_data_type': 'unknown',
 'dimension': 4,
 'num_items': 3,
 'knapsack_capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [16]:
node_coord

,Index,X,Y
0,1,0.0,0.0
1,2,4.0,0.0
2,3,8.0,3.0
3,4,0.0,3.0


In [17]:
profit_table=profit_matrix

In [18]:
import numpy as np

def calculate_distance_matrix(node_coord):
    coordinates = node_coord[['X', 'Y']].values
    x, y = np.meshgrid(coordinates[:, 0], coordinates[:, 1])

    distance_matrix = np.sqrt((x - x.T)**2 + (y - y.T)**2)
    
    return distance_matrix


In [19]:
distance_matrix=calculate_distance_matrix(node_coord)

In [20]:
distance_matrix

array([[0.        , 4.        , 8.54400375, 3.        ],
       [4.        , 0.        , 5.        , 5.        ],
       [8.54400375, 5.        , 0.        , 8.        ],
       [3.        , 5.        , 8.        , 0.        ]])

In [21]:
profit_table

,Index,Profit,Weight,Assigned_Node
0,2,34,30,2
1,3,40,40,3
2,4,25,21,4


In [26]:
packing_plan_example=[1,0,1]


In [27]:
profit_table['Picked'] = packing_plan_example


In [31]:
profit_table.to_clipboard()

In [22]:
pd.DataFrame(distance_matrix)

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [23]:
pd.DataFrame(calculate_distance_matrix(node_coord))

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [24]:
tour_example=[1,3,2,4]
packing_plan_example=[1,0,1]

In [80]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [88]:
def calculate_current_velocity(weight, capacity, min_speed, max_speed):
    if weight <= capacity:
        return max_speed - (weight / capacity) * (max_speed - min_speed)
    else:
        return min_speed


In [102]:
def calculate_cost(tour, packing_plan, variables,distance_matrix,profit_table):
    dimension = variables['dimension']
    max_speed = variables['max_speed']

    current_weight = 0
    total_time = 0
    current_velocity = max_speed

    for i in range(dimension):
        # Calculate the distance between consecutive cities
        distance = distance_matrix[tour[i % dimension]][tour[(i + 1) % dimension]]

        # Calculate the time taken to travel the distance
        time_to_travel = distance / current_velocity
        total_time += time_to_travel

        # Update current weight based on the packing plan
        current_weight += packing_plan[i % dimension] * profit_table['Weight'][i]

        # Calculate the current velocity
        current_velocity = calculate_current_velocity(current_weight, variables['capacity'],
                                                      variables['min_speed'], variables['max_speed'])

    return total_time


In [103]:
calculate_cost(tour=tour_example,packing_plan=packing_plan_example,variables=variables,distance_matrix=distance_matrix,profit_table=profit_table)

IndexError: index 4 is out of bounds for axis 0 with size 4

In [81]:
def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

In [82]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [85]:
calculate_cost(tour_example, packing_plan_example,
                                                variables['min_speed'],
                                                variables['max_speed'],
                                                variables['renting_ratio'],
                                                distance_matrix,
                                                profit_table)

KeyError: 0

In [ ]:
calculate_cost(tour_example, packing_plan_example,
                                                min_speed,
                                                max_speed,
                                                renting_ratio,
                                                distance_matrix,
                                                profit_table)

In [58]:
class TravellingThief:
    def __init__(self, population_size, dimension, num_items, capacity, min_speed, max_speed, renting_ratio, edge_weight_type, distance_matrix, profit_table):
        self.population_size = population_size
        self.dimension = dimension
        self.num_items = num_items
        self.capacity = capacity
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.renting_ratio = renting_ratio
        self.edge_weight_type = edge_weight_type
        self.distance_matrix = distance_matrix
        self.profit_table = profit_table

    @staticmethod
    def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

# Example usage
travelling_thief_instance = TravellingThief(population_size=4, dimension=280, num_items=279, capacity=25936,
                                            min_speed=0.1, max_speed=1.0, renting_ratio=5.61, edge_weight_type='CEIL_2D',
                                            distance_matrix=distance_matrix, profit_table=profit_matrix)

tour_example = [1, 3, 2, 4]
packing_plan_example = [1, 0, 1]

cost = travelling_thief_instance.calculate_cost(tour_example, packing_plan_example,
                                                travelling_thief_instance.min_speed,
                                                travelling_thief_instance.max_speed,
                                                travelling_thief_instance.renting_ratio,
                                                travelling_thief_instance.distance_matrix,
                                                travelling_thief_instance.profit_table)

print(f"Cost for the tour and packing plan: {cost}")


KeyError: 0

In [1]:
import numpy as np

# Given vector
genotype_vector = [0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

# Determine the length of the tour (n) and the length of the packing plan (m)
n = 4  # Assuming the tour has 4 cities
m = 3  # Assuming there are 3 items in the packing plan

# Split the genotype vector into two parts
tour_part = np.array(genotype_vector[:n-1])
packing_plan_part = np.array(genotype_vector[n-1:])

# Decode the tour part to a permutation vector
tour_permutation = np.argsort(tour_part) + 1
tour_permutation = np.insert(tour_permutation, 0, 1)  # Insert the starting city 1

# Decode the packing plan part to a binary decision vector
packing_decision_vector = (packing_plan_part > 0.5).astype(int)

# Display the results
print("Tour Phenotype:", tour_permutation)
print("Packing Plan Phenotype:", packing_decision_vector)


Tour Phenotype: [1 2 1 3]
Packing Plan Phenotype: [1 0 1]


In [2]:
        print(i,current_weight,current_velocity,distance,time_to_travel)


NameError: name 'variables' is not defined

In [13]:
packing_plan


NameError: name 'packing_plan' is not defined

In [32]:
profit_table

,Index,Profit,Weight,Assigned_Node,Picked
0,2,34,30,2,1
1,3,40,40,3,0
2,4,25,21,4,1


In [45]:
import numpy as np

n = 5  # Example value for n
vector_length = 2 * n - 1

unique_values = set()
while len(unique_values) < vector_length:
    value = np.random.uniform(low=0.0001, high=0.9999)  # Exclude 0 and 1
    unique_values.add(value)

vector = np.array(list(unique_values))


In [46]:
vector

array([0.88132891, 0.87708642, 0.91012356, 0.58740603, 0.82413326,
       0.91607612, 0.89317512, 0.91985402, 0.90248644])

In [47]:
[0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

[0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

In [53]:
sorted([0.5, 0.1, 0.8])

[0.1, 0.5, 0.8]

In [52]:
[x+1 for x in sorted([0.5, 0.1, 0.8])]

[1.1, 1.5, 1.8]

In [55]:
distance_matrix

array([[0.        , 4.        , 8.54400375, 3.        ],
       [4.        , 0.        , 5.        , 5.        ],
       [8.54400375, 5.        , 0.        , 8.        ],
       [3.        , 5.        , 8.        , 0.        ]])

In [64]:
import numpy as np

def get_tour(starting_index, vector):
    half_length = len(vector) // 2
    first_half = vector[:half_length]
    sorted_indices = np.argsort(first_half) + 2  # Adding 2 because of zero-indexing and to include the starting index
    sorted_indices = [starting_index] + sorted_indices.tolist()
    return sorted_indices

def get_packing_plan(vector):
    half_length = len(vector) // 2
    second_half = vector[half_length:]
    packing_plan = np.where(second_half > 0.5, 1, 0)
    return packing_plan

# Example vector
vector = np.array([0.5, 0.1, 0.8, 0.6, 0.1, 0.9])

# Get tour and packing plan
tour = get_tour(starting_index=1, vector=vector)
packing_plan = get_packing_plan(vector)

tour, packing_plan


([1, 3, 2, 4], array([1, 0, 1]))

In [68]:
"""3.15645915e-01 4.53030677e-01 2.66982264e-01 1.08928182e-01
 8.68166479e-01 6.29728518e-01 3.52518708e-01 6.75375994e-02
 6.26350592e-01 5.98660860e-01 9.22172511e-02 5.68515514e-02
 2.37132200e-01 2.35403670e-01 7.74371382e-01 4.26011311e-02
 3.54695279e-01 4.69166604e-01 7.67579920e-01 8.63516638e-01
 7.77620476e-01 2.25900156e-01 7.60389215e-01 4.41385371e-01
 2.17436735e-01 6.20251314e-01 1.41223510e-01 8.87551809e-02
 4.37561248e-01 2.89188192e-01 2.73069231e-01 5.49044754e-01
 9.65158649e-01 6.92406344e-01 6.44791747e-01 7.25829272e-01
 9.38300100e-01 4.72944714e-02 2.19055251e-03 3.46374663e-01
 6.77973870e-01 3.14115205e-01 7.83066568e-01 6.05550374e-01
 1.12060886e-01 8.86877968e-01 3.42211267e-01 5.51175913e-01
 8.15684753e-01 8.54537963e-01 6.27288426e-01 1.49496596e-01
 7.11852643e-01 3.20727456e-01 5.43364147e-01 5.89710407e-01
 2.31646917e-01 1.13270770e-02 7.28301435e-01 5.11450348e-02
 8.18167125e-01 2.45748710e-01 7.61276478e-02 5.95748028e-01
 1.81174414e-01 2.16680676e-01 5.12141298e-01 5.04229607e-02
 2.56736400e-01 2.54451891e-01 3.58113943e-01 5.04017540e-01
 1.51924379e-01 5.80728696e-01 1.89297018e-01 3.31209035e-01
 3.92400636e-01 4.22323896e-01 4.05886808e-01 5.84189356e-01
 9.91661443e-01 1.34504591e-01 5.07036061e-01 8.08908530e-01
 9.64719688e-01 7.94197165e-01 1.78611914e-01 8.66908031e-02
 8.24482422e-01 2.60644041e-01 9.29010603e-01 9.66595159e-01
 1.89505352e-01 9.64222991e-01 6.48914841e-01 2.38615183e-01
 8.01323980e-01 7.66065851e-01 2.11118690e-01 9.55127135e-02
 6.95051715e-01 4.38151648e-01 5.78623627e-01 7.16433736e-01
 3.52323404e-01 3.08884504e-01 6.76775372e-02 4.18771129e-01
 2.30360095e-01 3.06120981e-01 4.04072886e-01 9.39164440e-01
 1.80696464e-01 1.94442089e-01 4.56141281e-01 9.17005653e-01
 1.08160421e-01 3.87638323e-01 7.98300767e-02 3.98551588e-01
 8.75882461e-01 3.32068929e-01 6.55976158e-01 9.38786005e-01
 1.73427742e-01 6.11302447e-01 6.37563225e-02 1.24604124e-01
 4.15113517e-01 1.48866515e-01 8.85945533e-01 5.90106688e-01
 6.82844853e-01 1.00724154e-01 8.93409239e-01 3.66484876e-01
 8.49609675e-01 7.05659239e-01 1.29050950e-03 6.05378013e-01
 2.13653216e-02 4.14104420e-01 4.91170234e-02 9.92262017e-01
 4.68965468e-02 1.41739870e-01 2.51978438e-01 3.29358788e-01
 7.44883221e-01 1.92359500e-01 8.51710855e-01 4.78038301e-01
 9.90258943e-01 3.47459576e-01 5.36793631e-01 2.20791933e-01
 6.45009298e-01 6.42182598e-01 7.97079506e-01 5.69867527e-01
 3.40014060e-01 9.86790967e-01 5.89809869e-01 8.50221711e-01
 3.16317874e-01 8.12510068e-01 2.84337352e-01 6.61281715e-01
 1.14977087e-01 9.80158810e-01 8.01016205e-01 9.29012485e-01
 7.34405479e-01 3.14597571e-01 9.98558927e-01 5.16342607e-01
 3.45345671e-01 8.11041051e-01 2.86532991e-01 2.00982613e-01
 3.14126685e-01 4.00425839e-01 6.51925481e-01 3.03312085e-02
 1.87841761e-01 1.82841661e-01 6.76988919e-01 5.31352267e-01
 1.75215609e-01 4.58564305e-01 7.86987569e-01 3.72696090e-02
 1.08258582e-01 2.91632033e-01 8.52908069e-01 6.83483127e-01
 1.20654493e-02 7.75731306e-01 8.14850337e-01 9.58801565e-01
 9.44386062e-01 7.48724587e-01 8.84856369e-01 2.82872731e-01
 9.92240079e-01 5.99205886e-01 2.28333420e-01 8.03768880e-01
 4.63191207e-02 6.26781399e-01 9.83072718e-01 3.62678138e-01
 6.08837649e-01 7.64126395e-01 5.09902683e-01 9.35938626e-01
 2.10917501e-01 9.06093720e-01 9.49901307e-01 8.97181579e-01
 4.77335081e-01 7.31483075e-01 9.32738691e-02 4.40195197e-01
 1.51545131e-01 1.04916162e-01 5.96120573e-01 4.98548119e-01
 6.64062820e-01 2.39590315e-01 2.16716482e-01 1.41736214e-01
 4.56505724e-01 8.91101539e-02 6.67590593e-01 8.02963420e-01
 3.39177444e-01 9.62376868e-02 8.28363576e-01 5.13335897e-01
 4.90750566e-01 5.96443244e-01 4.65604625e-01 8.98784392e-01
 8.11255124e-01 9.28712208e-01 4.62498552e-01 6.07420663e-01
 1.90899655e-02 4.91951590e-01 7.44833721e-01 9.21511532e-01
 1.20502389e-01 5.35735046e-01 2.76740038e-01 7.75236759e-01
 2.97175424e-01 7.03760605e-01 7.89109852e-01 3.45365904e-01
 5.42818051e-01 9.43420213e-01 7.90262903e-01 6.37942023e-01
 9.90247924e-02 2.61989256e-01 5.59068121e-01 2.66564164e-01
 4.85505959e-01 7.47390673e-01 4.85974810e-02 9.57450384e-01
 7.52390126e-01 9.32010767e-01 7.22470776e-01 6.33382891e-01
 7.31623134e-01 9.23539207e-01 2.65950022e-01 7.25234929e-01
 8.00507244e-01 5.72518513e-02 2.13877257e-03 4.61449296e-01
 5.21796617e-01 6.58915855e-01 8.30846016e-01 2.97394184e-01
 9.34986090e-01 6.28809510e-01 5.57989306e-01 1.21002221e-01
 8.49239551e-01 8.29854917e-02 9.05292502e-02 6.22771833e-01
 4.94637474e-01 8.24057296e-01 1.13972098e-01 5.56995216e-01
 2.83437981e-01 6.41914147e-01 6.44857492e-01 8.70456480e-01
 7.82406385e-01 1.47704983e-01 3.81484492e-02 5.13978921e-01
 1.18726347e-01 6.18754663e-01 3.70116127e-01 1.85950111e-01
 1.50007515e-01 6.33492460e-02 2.05650702e-01 2.71988174e-01
 7.22715778e-01 6.04567967e-01 3.25186659e-01 5.93233484e-01
 3.90476607e-01 1.27194277e-01 3.37988711e-01 1.65549805e-01
 1.96151889e-01 2.93266118e-01 2.88382554e-01 3.29607535e-01
 5.94174916e-01 6.45579766e-01 1.21453031e-01 7.38913769e-01
 8.29360827e-01 9.47909597e-01 7.11188406e-01 7.52012877e-01
 1.25315063e-02 9.13103516e-01 4.74977294e-01 2.12328366e-02
 5.16698078e-01 9.61447379e-01 5.65671359e-01 8.43754013e-01
 4.47718378e-01 5.77610588e-01 6.92064926e-02 1.59911416e-01
 4.85992026e-01 6.67688081e-02 4.57901885e-01 5.26899899e-01
 4.25239388e-01 3.32207863e-01 2.37417891e-01 9.64139455e-01
 4.56645321e-01 3.08025899e-01 7.06525093e-01 6.76847377e-01
 1.44784628e-02 5.38852471e-02 3.36482026e-01 5.49102070e-01
 2.78151237e-01 3.74392853e-01 8.07930213e-01 9.49899667e-01
 2.67415338e-01 3.25924823e-01 3.68106366e-01 7.61264108e-01
 6.52760365e-01 5.21900179e-01 2.79261468e-01 7.40848063e-01
 4.31365469e-01 7.77981567e-01 7.35219357e-01 2.24699268e-01
 5.61289834e-01 7.31326179e-01 4.53864339e-02 2.37055857e-01
 2.78019479e-01 9.83152395e-01 7.23890479e-01 8.68315062e-01
 6.93867397e-01 3.98975093e-01 4.39029245e-01 4.79302533e-01
 5.63436829e-01 9.20689796e-01 2.79047556e-01 9.39431945e-04
 2.46275749e-01 8.11572188e-01 3.33844538e-01 3.26086177e-01
 8.47859279e-01 7.40246573e-02 4.55168324e-02 1.02198881e-01
 2.29364176e-01 8.72713417e-01 5.45853158e-01 6.72929191e-01
 6.35190148e-01 9.92156902e-01 5.73437943e-01 4.73507660e-01
 3.86937381e-01 6.20041853e-01 6.55725655e-01 5.45295840e-01
 3.73205671e-01 7.91417508e-01 3.66664915e-01 1.91092815e-02
 2.06762158e-01 2.15580940e-01 5.34511398e-01 8.14955668e-01
 1.03499853e-01 5.61093330e-01 5.36074730e-01 6.92525768e-01
 2.38251163e-01 7.15281859e-01 6.87795252e-01 9.13645099e-01
 6.40045121e-01 8.27926980e-01 1.30863254e-01 7.22187653e-01
 4.82151336e-01 7.65130587e-01 7.18553855e-01 5.05464969e-02
 9.03913806e-02 6.88558281e-01 9.25435381e-01 3.28666924e-01
 6.29965576e-01 7.68511786e-01 4.31066053e-01 2.83020703e-01
 5.07396036e-01 2.13237059e-01 4.28593501e-01 3.36404140e-01
 6.89877456e-01 9.37269972e-01 8.47230016e-01 6.77592491e-03
 3.90033396e-01 3.85674698e-01 1.62208461e-01 1.91527363e-01
 4.49462816e-01 1.16812187e-01 8.89536151e-01 2.56963068e-02
 3.35233227e-02 8.89456920e-01 5.01150749e-01 4.36671671e-01
 6.22809134e-01 5.00426624e-01 4.25458762e-01 1.81275554e-01
 6.17252571e-01 1.72958834e-01 3.53156654e-01 3.01425592e-01
 8.24294806e-01 1.06290612e-01 6.88192624e-01 1.18842918e-01
 3.77816220e-01 9.33151710e-01 8.45708492e-01 6.10373487e-01
 9.63766648e-01 6.00600567e-01 5.45146670e-01 3.26508158e-01
 6.30131410e-01 3.99116711e-01 9.42421544e-01 7.29800694e-01
 7.24192140e-01 7.48898090e-01 4.29362058e-01 7.07520237e-01
 2.03930339e-01 2.67544305e-01 5.72491066e-02 3.47733347e-02
 4.92959929e-04 7.50134429e-01 5.76271367e-01 5.83435666e-01
 5.93869680e-01 7.23725529e-01 6.29377010e-02 7.05168815e-01
 5.59570141e-01 6.06621568e-01 8.73148310e-01 1.14471243e-01
 6.13515504e-01 5.17464266e-01 1.92171724e-01 8.64569916e-01
 6.39381666e-01 9.28992832e-01 4.65240375e-02 5.73059203e-01
 6.96025236e-01 2.24000936e-01 4.08869526e-01 6.97422591e-01
 3.11093008e-02 6.09208374e-01 8.07310707e-01 6.14445666e-01
 4.97190147e-01 1.31616970e-02 9.48581375e-01 8.82429760e-01
 8.92307119e-01 9.46584796e-01 5.62527895e-01 4.38999303e-02
 5.47912848e-01 7.90657420e-01 4.40906883e-01 9.69686197e-02
 1.87603668e-01 9.46799108e-01 6.28377284e-01 9.32080443e-01
 9.25229036e-01 2.11718714e-01 6.71015967e-01 8.77004341e-01
 4.80702506e-01 8.92761913e-01 4.05982571e-01 1.58731198e-01
 8.61379884e-01 8.35415411e-01 2.58706997e-01 3.12324716e-01
 2.81289015e-02 6.83557537e-01"""

'3.15645915e-01 4.53030677e-01 2.66982264e-01 1.08928182e-01\n 8.68166479e-01 6.29728518e-01 3.52518708e-01 6.75375994e-02\n 6.26350592e-01 5.98660860e-01 9.22172511e-02 5.68515514e-02\n 2.37132200e-01 2.35403670e-01 7.74371382e-01 4.26011311e-02\n 3.54695279e-01 4.69166604e-01 7.67579920e-01 8.63516638e-01\n 7.77620476e-01 2.25900156e-01 7.60389215e-01 4.41385371e-01\n 2.17436735e-01 6.20251314e-01 1.41223510e-01 8.87551809e-02\n 4.37561248e-01 2.89188192e-01 2.73069231e-01 5.49044754e-01\n 9.65158649e-01 6.92406344e-01 6.44791747e-01 7.25829272e-01\n 9.38300100e-01 4.72944714e-02 2.19055251e-03 3.46374663e-01\n 6.77973870e-01 3.14115205e-01 7.83066568e-01 6.05550374e-01\n 1.12060886e-01 8.86877968e-01 3.42211267e-01 5.51175913e-01\n 8.15684753e-01 8.54537963e-01 6.27288426e-01 1.49496596e-01\n 7.11852643e-01 3.20727456e-01 5.43364147e-01 5.89710407e-01\n 2.31646917e-01 1.13270770e-02 7.28301435e-01 5.11450348e-02\n 8.18167125e-01 2.45748710e-01 7.61276478e-02 5.95748028e-01\n 1.81174